In [2]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
# import numpy as np

pd_jupiter = pd.read_excel('./jupyter.xlsx', engine='openpyxl')
pd_X = pd_jupiter[['x1', 'x2', 'x3']]
pd_Y = pd_jupiter.y
train_X, test_X, train_Y, test_y = train_test_split(pd_X, pd_Y, test_size=0.1, random_state=42)

treeCLF = RandomForestClassifier(random_state=0, min_samples_split=2, min_samples_leaf=1)
parameters = {
    'n_estimators': range(1, 5),
    'max_depth': range(3, 5),
}
cv = [(slice(None), slice(None))]

searchTree = GridSearchCV(treeCLF, parameters, cv=cv)
searchTree.fit(train_X, train_Y)
bestEstimatorTree = searchTree.best_estimator_
pred_y = bestEstimatorTree.predict(test_X)
test_y = test_y.reset_index(drop=True)
resultDatasetPredictions = pd.DataFrame({
    'test_y': test_y,
    'pred_y': pred_y,
    'diff': pred_y - test_y,
})
# resultDatasetPredictions
joblib.dump(bestEstimatorTree, './jupyter/jupyter.pkl')

['./jupyter/jupyter.pkl']

In [3]:



# and later you can load it
# clf = joblib.load('filename.pkl')

['./jupiter/jupiter.pkl']

,x1,x2,x3
0,1,3,51
